In [47]:
from asa_dsenum import generate_HNF, generate_superlattice, permutation
import numpy as np
import spglib
from copy import deepcopy

rutile = ([(4, 0, 0),(0, 4, 0),(0, 0, 3)],

          [(0, 0, 0),(0.5, 0.5, 0.5),(0.3, 0.3, 0.0),(0.7, 0.7, 0.0),(0.2, 0.8, 0.5),(0.8, 0.2, 0.5)],

          [14, 14, 8, 8, 8, 8])
o_sublattice = ([(4, 0, 0),(0, 4, 0),(0, 0, 3)],

                [(0.3, 0.3, 0.0),(0.7, 0.7, 0.0),(0.2, 0.8, 0.5),(0.8, 0.2, 0.5)],

                [8, 8, 8, 8])
parent_sym = spglib.get_symmetry(rutile)
HNF_list = generate_HNF.generate_all_superlattices(3)
reduced_HNF = generate_HNF.reduce_HNF_list_by_parent_lattice_symmetry(HNF_list, parent_sym["rotations"])

unique する前にsuperperiodic を省いてしまう方法をまずは試してみよう

In [22]:
def unique(parent_sym_jun, omomi4):
    """
    得られた究極の対称操作の辞書に基づいて、配列をユニークしていく

    parameters

    parent_sym_jun

    omomi4 は各空孔数ごとのcandidate集合

    retruns

    """
    omomi4_neo = deepcopy(omomi4)#copy wo sakusei
    lis = set()

    for i in range(len(omomi4)):#through all candidate
        if omomi4[i] in omomi4_neo:#kouho ga mada 生き残ってるかチェック
            
            for j in range(1, len(parent_sym_jun)):#置換操作について回す　
                d = dict()
                sta = ""
                for k in range(len(parent_sym_jun[0])):
                    d[parent_sym_jun[j][k]] = omomi4[i][k]#str辞書の作成
                for r  in range(len(parent_sym_jun[0])):#staの作成 sta is made from tikan[j]
                    sta += d[r]
                if int(sta) is not int(omomi4[i]):
                    if sta in omomi4_neo:
                        omomi4_neo.remove(sta)
                        lis.add(omomi4[i])
    return lis

In [25]:
def superperiodic_unique(trans_tikans, omomi4):
    omomi4_neo = deepcopy(omomi4)
    lis = set()
    
    for candidate in range(len(omomi4)):
        if omomi4[candidate] in omomi4_neo: #ある元の構造が生き残ってるかチェック
            id_superperiodic = False
            
            for permutation in range(1, len(trans_tikans)):#恒等操作以外の並進操作について回す
                d = dict()
                sta = ""
                for k in range(len(trans_tikans[0])):
                    d[trans_tikans[permutation][k]] = omomi4[candidate][k]
                for r in range(len(trans_tikans[0])):
                    sta += d[r]    # 置換によって生成した配列
                if int(sta) is not int(omomi4[i]):# 操作によって元とは別の配列になったかどうか
                    if sta in omomi4_neo:#まだ省いていない構造やったら
                        omomi4_neo.remove(sta)
                else:#作った配列と元の配列が一致、すなわちsuper_periodicやったら
                    id_superperiodic = True
            if  not id_superperiodic:
                    lis.add(omomi4[candidate])
                    
    lis = list(lis)
    return lis
                    
                    
                    
                    

In [117]:
index = 2
hoge = 0

HNF_list = generate_HNF.generate_all_superlattices(index)
reduced_HNF = generate_HNF.reduce_HNF_list_by_parent_lattice_symmetry(HNF_list, parent_sym["rotations"])

parent_lattice =  generate_superlattice.get_superlattice(rutile, reduced_HNF[hoge], index)
o_sublattice = generate_superlattice.get_o_superlattice(rutile, reduced_HNF[hoge], index)
set_of_translations = permutation.gene_trans(reduced_HNF[hoge])
dic_zahyou = dict()
for i in range(len(o_sublattice[1])):
    dic_zahyou[i] = o_sublattice[1][i]
set_of_transtikans = permutation.get_trans_perms(dic_zahyou, set_of_translations)

goalen = permutation.shin_get_permutation(o_sublattice, parent_sym, reduced_HNF[hoge])
omomi4 = permutation.make_candidate(o_sublattice,2) 
ru = superperiodic_unique(set_of_transtikans, omomi4)
asa = unique(goalen, omomi4)
ru

00010001
00100010
01000100
10001000


['00010010',
 '00000011',
 '00001010',
 '00001001',
 '00011000',
 '00100100',
 '01001000',
 '00000110',
 '00001100',
 '00000101',
 '00101000',
 '00010100']

"00100010"っていうsuperperiodicな配列が残ってる
ある配列が与えられたset_of_transtikansの下でsuperperiodicでないかを調べる関数がほしい

In [118]:
def superperiodic_unique(trans_tikans, omomi4):
    omomi4_neo = deepcopy(omomi4)
    lis = set()
    
    for candidate in range(len(omomi4)):
        if omomi4[candidate] in omomi4_neo: #ある元の構造が生き残ってるかチェック
            id_superperiodic = False
            
            for permutation in range(1, len(trans_tikans)):#恒等操作以外の並進操作について回す
                d = dict()
                sta = ""
                for k in range(len(trans_tikans[0])):
                    d[trans_tikans[permutation][k]] = omomi4[candidate][k]
                for r in range(len(trans_tikans[0])):
                    sta += d[r]    # 置換によって生成した配列
                if int(sta) == int(omomi4[candidate]):# 作った配列と元の配列が一致、すなわちsuper_periodicやったら
                    print(omomi4[candidate])
                    id_superperiodic = True
                else:#操作によって元とは別の配列になったかどうか
                    if sta in omomi4_neo:#まだ省いていない構造やったら
                        omomi4_neo.remove(sta)
                        
            if not id_superperiodic:
                lis.add(omomi4[candidate])
                    
    lis = list(lis)
    return lis
                    
                    
                    
                    

In [119]:
ru2 = ['00010010','01000100']

In [124]:
asa = list(asa)

In [125]:
ru2 = superperiodic_unique(set_of_transtikans, asa)

00010001


In [126]:
ru2

['00010010', '00000011', '00000101']

In [68]:
a = True
if not a:
    print(1)
else:
    print(0)

0


In [99]:
def effect_permutation(permutation, str_array):
    d = dict()
    sta = ""
    for k in range(len(permutation)):
        d[permutation[k]] = str_array[k]
    for r in range(len(permutation)):
        sta += d[r]    # 置換によって生成した配列
    return sta

In [103]:
mori = effect_permutation(set_of_transtikans[1], ru2[0])

In [104]:
mori

'00100001'

In [106]:
if int(mori) == int(ru2[1]):# 操作によって元とは別の配列になったかどうか
    print("sp")
else:
    print("not_sp")

not_sp


In [93]:
a = int(mori)

In [94]:
b = int(ru2[1])

In [95]:
if a == b:
    print(0)

0


In [109]:
if not 3 == 4:
    print(4)

4
